In [30]:
import csv
import pandas as pd
import numpy as np

df = pd.read_csv('6_diabetes.csv')
df = df.astype(float)
train = df.sample(frac=0.8,random_state=400)
test = df.drop(train.index)

In [31]:
outcomegroup = train.groupby(df.columns[-1])
n_attr = len(df.columns)-1
print(n_attr)
summaries = {}

for classValue, instances in outcomegroup:
    attr=[]
    mean = list(instances.mean(axis=0).values)
    stdev = list(instances.std(axis=0).values)
    for i in range(n_attr):
        attr.append([mean[i], stdev[i]])
    summaries[classValue] = attr
    

8


In [32]:
import math

def calculateprob(x,mean,std):
    exponent = np.exp(-math.pow((x-mean),2)/2*math.pow(std,2))
    return (1/(math.sqrt((2*math.pi)*math.pow(std,2))))*exponent

def calculateClassProb(summaries, x_vec):
    probabilities = {}
    for classvalue, classsummaries in summaries.items():
        probabilities[classvalue] = 1
        for i in range(len(classsummaries)):
            x = x_vec[i]
            mean,std = classsummaries[i]
            prob = calculateprob(x,mean,std)
            probabilities[classvalue] *= prob
    return probabilities

def Predict(summaries,x_vec):
    prob = calculateClassProb(summaries,x_vec)
    bestlabel, bestvalue = None, -1
    for classvalue,probability in prob.items():
        if bestlabel is None or probability > bestvalue:
            bestlabel = classvalue
            bestvalue = probability
    return bestlabel
    

In [33]:
predictions = []
testset = test.values.tolist()
for i in range(len(test)):
    result = Predict(summaries,testset[i])
    predictions.append(result)

def getAccuracy(test,predictions):
    correct = 0
    for i in range(len(test)):
        if test.iloc[i,-1] == predictions[i]:
            correct+=1
    return (correct/len(test))

print(getAccuracy(test, predictions))

0.7012987012987013
